In [0]:
%run ../config/project_config

In [0]:
from pyspark.sql.functions import col

In [0]:
target_table = f"{CATALOG_NAME}.{SCHEMA_BRONZE}.county_metrics"

print(f"Testing Table: {target_table}")

try:
    df = spark.read.table(target_table)
    
    # --- TEST 1: DATA EXISTENCE ---
    count = df.count()
    print(f"Row Count: {count}")
    assert count > 0, f"❌ FAILURE: Table {target_table} is empty!"
    print("✅ TEST 1 PASSED: Data exists.")

    # --- TEST 2: AUDIT COLUMNS ---
    columns = df.columns
    assert "load_dt" in columns, "❌ FAILURE: Column 'load_dt' is missing!"
    assert "source_file" in columns, "❌ FAILURE: Column 'source_file' is missing!"
    print("✅ TEST 2 PASSED: Audit columns exist.")

    # --- TEST 3: AUDIT DATA QUALITY ---
    null_audit_count = df.filter(col("load_dt").isNull() | col("source_file").isNull()).count()
    assert null_audit_count == 0, f"❌ FAILURE: {null_audit_count} rows have NULL audit values."
    print("✅ TEST 3 PASSED: Audit columns are populated.")

    # --- TEST 4: Lineage Check ---
    invalid_source_count = df.filter(~col("source_file").contains(".csv")).count()
    assert invalid_source_count == 0, f"❌ FAIL: {invalid_source_count} rows have invalid source filenames."
    print("✅ TEST 4 PASSED: Source file is csv.")

    print("\n ALL UNIT TESTS PASSED SUCCESSFULLY.")

except Exception as e:
    print(f"\n❌ CRITICAL TEST FAILURE: {e}")
    raise e

Testing Table: Zillow.bronze.county_metrics
Row Count: 259395
✅ TEST 1 PASSED: Data exists.
✅ TEST 2 PASSED: Audit columns exist.
✅ TEST 3 PASSED: Audit columns are populated.
✅ TEST 4 PASSED: Source file is csv.

 ALL UNIT TESTS PASSED SUCCESSFULLY.
